In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

import json
import ast

In [2]:
url = "https://ocrportal.hhs.gov/ocr/breach/breach_report.jsf"

response = requests.get(url)

# create a BeautifulSoup object and fill the content from our url
soup = BeautifulSoup(response.content, 'html.parser')
soup.title.string

matches = soup.find("div", class_='ui-datatable-tablewrapper')


# collect raw rows from table
rows = matches.findAll("tr")

# header attributes of the table
headers = [th.text.rstrip() for th in rows[0].find_all('th')]


# extract all rows from table
allRows = []

for row in rows:
    data = [td.text.rstrip() for td in row.find_all('td')]
    
    allRows.append(data)
allRows = allRows[1:-4]

# create a dataframe from collected data
pd.DataFrame(allRows, columns=headers)

,Expand All,Name of Covered Entity,State,Covered Entity Type,Individuals Affected,Breach Submission Date,Type of Breach,Location of Breached Information,Business Associate Present,Web Description
0,,Fertility Specialists Medical Group,CA,Healthcare Provider,9437,05/15/2023,Hacking/IT Incident,Network Server,No,
1,,Lehigh Valley Health Network,PA,Healthcare Provider,627,05/15/2023,Hacking/IT Incident,Network Server,No,
2,,R&B Corporation of Virginia d/b/a Credit Contr...,VA,Business Associate,345523,05/13/2023,Hacking/IT Incident,Network Server,Yes,
3,,PharMerica Corporation,KY,Healthcare Provider,5815591,05/12/2023,Hacking/IT Incident,Network Server,No,
4,,Great Expressions Dental Centers,MI,Healthcare Provider,528,05/12/2023,Hacking/IT Incident,Network Server,No,
...,...,...,...,...,...,...,...,...,...,...
91,,Atlantic Dialysis Management Services - Spring...,NY,Business Associate,922,03/22/2023,Hacking/IT Incident,Network Server,Yes,
92,,Atlantic Dialysis Management Services - Ridgew...,NY,Business Associate,2204,03/22/2023,Hacking/IT Incident,Network Server,Yes,
93,,Atlantic Dialysis Management Services - 522012,NY,Business Associate,563,03/22/2023,Hacking/IT Incident,Network Server,Yes,
94,,Atlantic Dialysis Management Services - Newtow...,NY,Business Associate,970,03/22/2023,Hacking/IT Incident,Network Server,Yes,


## Convert JSON

In [43]:
df = pd.read_csv('https://raw.githubusercontent.com/TatKhachatryan/current-report/main/breach_report_current.csv')
df = df[df.columns[:-1]]
df

,Name of Covered Entity,State,Covered Entity Type,Individuals Affected,Breach Submission Date,Type of Breach,Location of Breached Information,Business Associate Present
0,Fertility Specialists Medical Group,CA,Healthcare Provider,9437,05/15/2023,Hacking/IT Incident,Network Server,No
1,Lehigh Valley Health Network,PA,Healthcare Provider,627,05/15/2023,Hacking/IT Incident,Network Server,No
2,R&B Corporation of Virginia d/b/a Credit Contr...,VA,Business Associate,345523,05/13/2023,Hacking/IT Incident,Network Server,Yes
3,Illinois Department of Healthcare and Family S...,IL,Health Plan,50839,05/12/2023,Hacking/IT Incident,Network Server,No
4,Solutran,MN,Business Associate,17728,05/12/2023,Hacking/IT Incident,Network Server,Yes
...,...,...,...,...,...,...,...,...
860,Walgreen Co.,IL,Healthcare Provider,6498,05/28/2021,Improper Disposal,Paper/Films,No
861,LogicGate,IL,Business Associate,47035,05/26/2021,Hacking/IT Incident,Network Server,Yes
862,Buckeye Health Plan,OH,Health Plan,2334,05/26/2021,Hacking/IT Incident,Network Server,Yes
863,"Gastroenterology Consultants, PA",TX,Healthcare Provider,161698,03/19/2021,Hacking/IT Incident,Network Server,No


## Convert rows to JSON

In [44]:
# the following loop will create separate json files for each row in the table

# for i in data.index:
#     data.loc[i].to_json("row{}.json".format(i))

## Read JSON

In [45]:
dicts = df.to_dict(orient='records')
json_1 = dicts[0]

json_check = json.dumps(json_1)

print(json_check)

{"Name of Covered Entity": "Fertility Specialists Medical Group", "State": "CA", "Covered Entity Type": "Healthcare Provider", "Individuals Affected": 9437, "Breach Submission Date": "05/15/2023", "Type of Breach": "Hacking/IT Incident", "Location of Breached Information": "Network Server", "Business Associate Present": "No"}


## Data Validation

In [46]:
def validateJSON(jsonData):
    try:
        json.loads(jsonData)
    except ValueError as err:
        return False
    return True


print("Given JSON string is Valid", validateJSON(json_check))

Given JSON string is Valid True
